In [1]:
# 模型重新训练和保存
from time import time
import os
import tensorflow as tf
import numpy as np
import VGG16_model as model
import utils

![函数列表](functions.PNG)

In [ ]:
startTime = time()
batch_size = 32
capacity = 256 # 内存中存储的最大数据容量
means = [123.68, 116.779, 103.939] # VGG训练时图像预处理所减均值 rgb三通道

xs,ys = utils.get_file('./data/train/') # 获取图像列表和标签列表

# 通过读取列表来载入批量图片及标签
image_batch, label_batch = utils.get_batch(xs, ys, 224, 224, batch_size, capacity) 

x = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None,2])

vgg = model.vgg16(x)
fc8_finetuining = vgg.probs # 即softmax(fc8)

loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc8_finetuining, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_function)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
vgg.load_weights('../vgg16_weights.npz', sess)

saver = tf.train.Saver()




-----------weights loaded---------------


In [ ]:
# 启动线程
coord = tf.train.Coordinator() # 使用协调器Coordinator 来管理线程
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

epoch_start_time = time()

# 迭代训练 （教学这里是1000轮。。。他机器每次0.2s一轮我不知道咋搞的，反正我这里要25s一轮。。。跑了四小时）
for i in range(1000):
    images , labels = sess.run([image_batch, label_batch])
    labels = utils.onehot(labels) # 用one-hot 形式对标签进行编码
    
    sess.run(optimizer, feed_dict={x:images, y:labels})
    loss = sess.run(loss_function, feed_dict={x:images, y:labels})
    print('Now the loss is %f'%loss)
    
    epoch_end_time = time()
    print('Current epoch takes: ', (epoch_end_time - epoch_start_time))
    
    epoch_start_time = epoch_end_time
    if (i + 1) % 10 == 0:
        saver.save(sess,os.path.join('./model/','epoch {:06d}.ckpt'.format(i)))
    
    print("--------Epoch %d is finisthed------------"%i)

# 模型保存
saver.save(sess, './model/')
print('Optimization Finished')

duration = time() - startTime
print('Train finishied takes: duration', '{:.2f}'.format(duration))

# 关闭线程
coord.request_stop()# 通知其他线程关闭
coord.join(threads) # join操作等待其他线程结束，其他所有线程关闭后，这一函数才能返回


Now the loss is 0.595216
Current epoch takes:  26.62547779083252
--------Epoch 0 is finisthed------------
